# Example Brick Model for process
Example brick model will be 2 AHUs, 60 terminal units.
Half are for perimeter zones vavs with reheat, half are for core zones and are cooling only 

10 core zone terminal units are for mechanical rooms, networking closets, and electrical rooms that must be served 24/7

In [ ]:
from rdflib import Namespace, URIRef, RDFS
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Model, Library
from buildingmotif.namespaces import BMOTIF, BRICK
from buildingmotif.model_builder import TemplateBuilderContext

: 

In [ ]:
BLDG = Namespace("urn:example#")
bm = BuildingMOTIF("sqlite://", "topquadrant")
bldg = Model.create("urn:example#")
bldg.graph.bind('',BLDG)
brick = Library.load(ontology_graph="Brick.ttl")

In [ ]:
brick_tmpl = Library.load(directory='brick-templates')

In [ ]:
# start with the tank. This instantiates the template with 'my_tank' as the name
# You can assign other parameters here as well. If the value you give to a parameter
# is a string (e.g. 'my_tank') then the builder will automatically make a URL with the BLDG (see cell above) as the namespace
# ahu = ctx['multiple-zone-ahu'](name='ahu1')
# # now create an instance of the water flow property
# flow_prop = ctx['water-flow'](name='tank_outlet_water_flow')
# flow_prop['name-owner'] = my_tank['out']
# # now create the sensor
# flow_sensor = ctx['sensor'](name='my_flow_sensor')
# flow_sensor['where'] = my_tank['out'] # the sensor measures at the outlet of the tank
# flow_sensor['property'] = flow_prop['name'] # associate the sensor's "property" param with the flow property

In [ ]:
# bldg_config
# assuming cooling+only and reheat vavs split evenly. Computer spaces are all in the cooling only vavs.
# [ahus, cooling_only_vavs_per_ahu, reheat_vavs_per_ahu]
# Could add amt computer spaces_per_ahu to building if it serves example later

# the context helps us build up our model
ctx = TemplateBuilderContext(BLDG)
ctx.add_templates_from_library(brick)
ctx.add_templates_from_library(brick_tmpl)

In [ ]:
bldg_config = {'ahus': 2, 'cv': 10, 'hv': 10}
bldg_ctx_dict = {}
for i in range(bldg_config['ahus']):
    ahu_name = f'ahu_{i}'
    # bldg_ctx_dict[name] = ctx['multiple-zone-ahu'](name=name)
    ahu = ctx['multiple-zone-ahu'](name=ahu_name)
    for cv in range(bldg_config['cv']):
        cv_name = f'cvav_{i}_{cv}'
        cvav = ctx['vav-cooling-only'](name=cv_name)
        ctx['feeds'](name = ahu['name'], target = cvav['name'])
    for hv in range(bldg_config['hv']):
        hv_name = f'hvav_{i}_{hv}'
        hvav = ctx['vav-with-reheat'](name=hv_name)
        ctx['feeds'](name = ahu['name'], target = hvav['name'])

In [ ]:
bldg.add_graph(ctx.compile())

In [ ]:
print(bldg.graph.serialize(format='ttl'))

In [ ]:
bldg.graph.serialize('brick-example.ttl', format = 'ttl')